In [ ]:
from src.conexion.conexion_singleton import  ConexionSingleton
from sqlalchemy import text
from sql.consultas_sql import ejecutar_consulta
import logging

In [2]:
con = ConexionSingleton()
session = con.get_session()

2025-06-06 17:28:06,425 INFO sqlalchemy.engine.Engine SELECT DATABASE()


2025-06-06 17:28:06,425 - INFO - SELECT DATABASE()


2025-06-06 17:28:06,428 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-06-06 17:28:06,428 - INFO - [raw sql] {}


2025-06-06 17:28:06,430 INFO sqlalchemy.engine.Engine SELECT @@sql_mode


...

2025-06-06 17:28:06,432 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names


2025-06-06 17:28:06,432 - INFO - SELECT @@lower_case_table_names


2025-06-06 17:28:06,433 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-06-06 17:28:06,433 - INFO - [raw sql] {}


conexión ok.


In [3]:

logger = logging.getLogger('sqlalchemy.engine.Engine')
logger.setLevel(logging.ERROR)   
logger.propagate = False      

for handler in logger.handlers[:]:
    logger.removeHandler(handler)


In [10]:
ventas_por_categoria = '''  
SELECT 
    c.categoryname AS CategoryName,
    COUNT(s.salesid) AS total_ventas,
    SUM(s.totalprice) AS total_ingresos
FROM sales s
JOIN products p ON s.productid = p.productid
JOIN categories c ON p.categoryid = c.categoryid
GROUP BY c.categoryname
ORDER BY total_ventas DESC;


'''

In [13]:
arm = text(ventas_por_categoria)
result = session.execute(arm)
print(result.keys())
for row in result:
    print(row)

RMKeyView(['CategoryName', 'total_ventas', 'total_ingresos'])
('Confections', 6299, Decimal('998526.00'))
('Meat', 5486, Decimal('862283.00'))
('Poultry', 5156, Decimal('815645.00'))
('Cereals', 5110, Decimal('805381.00'))
('Produce', 4771, Decimal('736564.00'))
('Beverages', 4196, Decimal('673480.00'))
('Snails', 4119, Decimal('644595.00'))
('Seafood', 4059, Decimal('642391.00'))
('Shell fish', 3949, Decimal('621162.00'))
('Dairy', 3892, Decimal('602614.00'))
('Grain', 2963, Decimal('460978.00'))


In [ ]:
top_clientes = '''  
SELECT
    cu.CustomerID as cliente,
    cu.FirstName nombre_cliente,
    cu.LastName apellido_cliente,
    COUNT(s.SalesID) AS cantidad_compras,
    SUM(s.TotalPrice) AS total_gastado
FROM Sales s
JOIN Customers cu ON s.CustomerID = cu.CustomerID
GROUP BY cu.CustomerID, cu.FirstName, cu.LastName
ORDER BY total_gastado DESC
LIMIT 10;


'''

In [42]:
arm = text(top_clientes)
result = session.execute(arm)
print(result.keys())
for row in result:
    print(row)

RMKeyView(['CustomerID', 'FirstName', 'LastName', 'CantidadCompras', 'TotalGastado'])
(91038, 'Darcy', 'Bullock', 4, Decimal('1872.00'))
(94115, 'Blake', 'Dalton', 5, Decimal('1848.00'))
(96485, 'Forrest', 'Morton', 3, Decimal('1700.00'))
(98273, 'Curtis', 'Harmon', 4, Decimal('1650.00'))
(73360, 'Allison', 'Davies', 5, Decimal('1615.00'))
(79369, 'John', 'Gross', 4, Decimal('1491.00'))
(97705, 'Fred', 'Roberts', 3, Decimal('1475.00'))
(95885, 'Aimee', 'Banks', 4, Decimal('1475.00'))
(94696, 'Spencer', 'Booker', 5, Decimal('1440.00'))
(89007, 'Jolene', 'Vincent', 4, Decimal('1426.00'))


In [8]:
query = ''' 
with producto_ciudad_resumen as (
    select
        ci.cityname as ciudad,
        p.productname as producto,
        avg(p.price) as precio_promedio,
        count(sa.salesid) as cantidad_ventas
    from sales sa
    join customers cu on sa.customerid = cu.customerid
    join cities ci on cu.cityid = ci.cityid
    join products p on sa.productid = p.productid
    group by ci.cityname, p.productname
),
ventas_totales_ciudad as (
    select
        ciudad,
        sum(cantidad_ventas) as total_cant
    from producto_ciudad_resumen
    group by ciudad
)
select
    ps.ciudad as nombre,
    ps.producto,
    ps.precio_promedio,
    ps.cantidad_ventas,
    round((cast(ps.cantidad_ventas as decimal) / vt.total_cant) * 100, 2) as porcentaje_ventas_ciudad,
    rank() over (partition by ps.ciudad order by ps.precio_promedio desc) as ranking_precio_producto
from producto_ciudad_resumen ps
join ventas_totales_ciudad vt on ps.ciudad = vt.ciudad
order by ps.ciudad, ranking_precio_producto;
'''

In [12]:
arm = text(query)
result = session.execute(arm)
print(result.keys())
for row in result:
    print(row)

RMKeyView(['nombre', 'producto', 'precio_promedio', 'cantidad_ventas', 'porcentaje_ventas_ciudad', 'ranking_precio_producto'])
('Akron', 'Puree - Passion Fruit', Decimal('98.82630000'), 1, Decimal('0.20'), 1)
('Akron', 'Bread - Calabrese Baguette', Decimal('98.59780000'), 1, Decimal('0.20'), 2)
('Akron', 'Lettuce - Treviso', Decimal('96.41270000'), 2, Decimal('0.40'), 3)
('Akron', 'Hot Chocolate - Individual', Decimal('96.08230000'), 1, Decimal('0.20'), 4)
('Akron', 'Pork - Hock And Feet Attached', Decimal('95.74150000'), 2, Decimal('0.40'), 5)
('Akron', 'Pop Shoppe Cream Soda', Decimal('95.73870000'), 2, Decimal('0.40'), 6)
('Akron', 'Scampi Tail', Decimal('95.09570000'), 2, Decimal('0.40'), 7)
('Akron', 'Pail For Lid 1537', Decimal('94.62530000'), 1, Decimal('0.20'), 8)
('Akron', 'Beef - Rib Eye Aaa', Decimal('94.06640000'), 2, Decimal('0.40'), 9)
('Akron', 'Wasabi Powder', Decimal('93.97520000'), 1, Decimal('0.20'), 10)
('Akron', 'Wine - Gato Negro Cabernet', Decimal('93.80470000'),